In [2]:
import sklearn

In [3]:
import minsearch

In [4]:
import json

In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [9]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)


In [10]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [17]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [210]:
q = "the course has already started, can I still enroll?"

In [20]:
index.fit(documents)

In [28]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict= {'course': 'data-engineering-zoomcamp'},
    boost_dict = boost, 
    num_results=5
)

In [29]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [36]:
from openai import OpenAI

In [208]:
access_key_json = json.load(open("./access_keys/access_key.json"))

In [209]:
client = OpenAI(api_key=access_key_json["OPENAI_ACCESS_KEY"])

In [211]:
response = client.chat.completions.create(
    model='gpt-4o', 
    messages=[{"role": "user","content": q}]
)

In [212]:
response.choices[0].message.content

"That depends on the specific course and the policies of the institution offering it. Here are some steps you can take to find out:\n\n1. **Check the Enrollment Deadline**: Review the course or institution's website for any information on enrollment deadlines. Some courses allow late enrollments with a certain grace period.\n   \n2. **Contact the Instructor**: Reach out to the instructor or course coordinator directly to ask if it's still possible to join the class. They might make exceptions or inform you about the possibility of catching up on missed material.\n\n3. **Institution Policies**: Look into the institution's policies on late enrollment. Some schools might have formal procedures for adding a course after it has started.\n\n4. **Catch-Up Plan**: Be prepared to demonstrate how you'll catch up on any missed work or lectures. Showing initiative and a plan might make the instructor more willing to accommodate a late add.\n\n5. **Look for Alternatives**: If late enrollment isn't 

In [49]:
prompt_template = """
You're a course teaching assistan. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering the quesiton. 
If the CONTECT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTET: {context}
"""
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [50]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [57]:
prompt = prompt_template.format(question=q, context=context).strip()

In [58]:
print(prompt)

You're a course teaching assistan. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering the quesiton. 
If the CONTECT doesn't contain the answer, output NONE.

QUESTION: the course has already started, can I still enroll?

CONTET: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.



In [59]:
response = client.chat.completions.create(
    model='gpt-4o', 
    messages=[{"role": "user","content": prompt}]
)

In [61]:
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [113]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict= {'course': 'data-engineering-zoomcamp'},
        boost_dict = boost, 
        num_results=5
    )

    return results

In [173]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistan. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering the quesiton. 

QUESTION: {question}

CONTET: {context}
""".strip()
    
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [163]:
def llm(prompt):
    response = client.chat.completions.create(
    model='gpt-4o', 
    messages=[{"role": "user","content": prompt}]
    )
    return response.choices[0].message.content

In [178]:
def rag(query):
    query = query
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [179]:
rag('the course has already started, can I still enroll?')

"Yes, even if you don't register, you're still eligible to submit the homework. However, be aware that there will be deadlines for turning in the final projects, so make sure not to leave everything for the last minute."

In [180]:
from elasticsearch import Elasticsearch

In [181]:
es_client = Elasticsearch('http://127.0.0.1:9200')

In [183]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
            }
        }
    }

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [185]:
from tqdm.auto import tqdm

In [186]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [188]:
query = 'the course has already started, can I still enroll?'

In [197]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [198]:
response = elastic_search(query)

In [199]:
response

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [200]:
def rag(query):
    query = query
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [201]:
rag(query)

"Yes, you can still join the course even after it has started. You are eligible to submit homeworks, but be aware of the deadlines for turning in the final projects. Don't leave everything until the last minute."

In [202]:
import bitsandbytes

ModuleNotFoundError: No module named 'bitsandbytes'

zsh:1: command not found: nvidia-smi
